<a href="https://colab.research.google.com/github/geoimaginarte/eol_python/blob/master/tarea_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prueba de la app Plantnet en la identificacion de especies

Creado por: Jose Francisco Núñez Obando y Maria Laura Pizarro<br>

El siguiente documento corresponde con una evaluación del porcentaje de precisión en la identificación de especies de plantas mediante el uso de la aplicación Plantnet.<br>
<br>
Para cumplir con este objetivo se descargan una serie de fotografia desde los sitios GBIF y EOL usando sus api's respectivas, las cuales posteriormente se utilizan como objetos de prueba para la identificacion de especies en la aplicacion de PlantNet que se encuentra en linea, siendo esta la misma que se encuentra en formato de app para android y ios. Al respecto de la puesta a prueba de dicho servicio de identificacion, se planteo el uso de la técnica de webscrapping utilizando para ello el webdriver de 'chrome' y la librería de Python llamada Selenium, además de otras librerías que complementan el análisis.<br>
<br>En los siguientes apartados se describen los procedimientos y se anotan las partes de codigo utilizadas para llavar a cabo cada proceso.
<br><br>
A continuación, algunas de las librerias python utilizadas inicialmente para completar con los primeros procedimientos:

In [35]:
import requests
import urllib.request
from lxml import html, etree
import json
from pandas.io.json import json_normalize
#import squarify
import warnings
warnings.filterwarnings('ignore')
import time
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from bs4 import BeautifulSoup

## Descarga de datos usando la API de GBIF

En esta sección se procede a realizar la consulta a la API de GBIF en la que únicamente se establecen como parámetros el pais y el reino.

In [75]:
parameters = {"country": "CR",
              "limit":300,
              "scientificname" : "Plantae"
              }
url = "https://api.gbif.org/v1/occurrence/search?"

q_gbif = requests.request(method="get", url=url, params=parameters)
j_gbif = json.loads(q_gbif.text)
L = json_normalize(j_gbif['results'])
L = L[['kingdom', 'phylum','order', 'family', 'genus', 'species', 'scientificName']]

Una vez se obtienen los resultados de la consulta, se procedio a realizar una selección de 20 muestras aleatorias del total.

In [76]:
L2 = L.sample(n=20, random_state=1)
L2

,kingdom,phylum,order,family,genus,species,scientificName
189,Plantae,Tracheophyta,Fabales,Fabaceae,Senna,Senna alata,Senna alata (L.) Roxb.
123,Plantae,Tracheophyta,Cucurbitales,Cucurbitaceae,Momordica,Momordica charantia,Momordica charantia L.
185,Plantae,Tracheophyta,Arecales,Arecaceae,Cocos,Cocos nucifera,Cocos nucifera L.
213,Plantae,Tracheophyta,Asterales,Asteraceae,Sphagneticola,Sphagneticola trilobata,Sphagneticola trilobata (L.) Pruski
106,Plantae,Tracheophyta,Zingiberales,Heliconiaceae,Heliconia,Heliconia latispatha,Heliconia latispatha Benth.
127,Plantae,Tracheophyta,Lamiales,Acanthaceae,Aphelandra,Aphelandra scabra,Aphelandra scabra (Vahl) Sm.
176,Plantae,Tracheophyta,Piperales,Piperaceae,Peperomia,Peperomia pittieri,Peperomia pittieri C.DC. ex T.Durand & Pittier
73,Plantae,Tracheophyta,Malpighiales,Hypericaceae,Vismia,Vismia macrophylla,Vismia macrophylla Kunth
275,Plantae,Tracheophyta,Ericales,Primulaceae,Clavija,Clavija biborrana,Clavija biborrana Oerst.
242,Plantae,Tracheophyta,Gentianales,Rubiaceae,Hamelia,Hamelia patens,Hamelia patens Jacq.


## Consulta a la API de Encyclopedia of Life (EOL)

Con el fin de obtener una cantidad de 3 fotos de cada una especie de la muestra, se realiza una consulta a la API de EOL desde la que se obtiene una referencia a una subpagina de la especie correspondiente. En dicha subpagina se consulta de forma automatizada la seccion de media en la que se encuentran las imagenes por especie, por lo que para una descarga de las misma se emplea una tecnica de web scrapping. Al final se obtiene un dataframe que contiene el nombre de especie y una URL de las fotografias asociadas a esta. 

In [87]:
L3=L2.groupby(['species'])['species'].head(1).reset_index(drop=True)
url_eol = "http://eol.org/api/search/1.0.json?"
df = pd.DataFrame(columns=('specie', 'url'))

In [88]:
species_count=0
rows_list = []
for i in L3:
    #EOL DATA
    parameters = {"q": i}
    q_eol = requests.request(method="get", url=url_eol, params=parameters)
    j_eol = json.loads(q_eol.text)
    
    #Se filtran los datos que tengan mas de 3 resultados 
    jd_eol = json_normalize(j_eol['results'])
    rows_temp = []  
    #Se recorren los links de los resultados y se evalua si el link devuelve informacion
    for link in j_eol['results']:
        dict1 = {}
        r = requests.get(link['link']+'/media')
        html = r.text
        soup = BeautifulSoup(html, 'lxml')
        links = soup.find_all('div', {'class': 'js-grid-modal-toggle uk-card-media-top uk-inline-clip uk-transition-toggle'})
        
        for y in links:
            dict1.update({'Specie':i, 'Url':y.find('img')['src']})
            rows_temp.append(dict1) 
            if(len(rows_temp) == 3):
                species_count+=1
                rows_list += rows_temp
                break
        if(len(rows_temp) == 3):
            break
    #Se detiene el ciclo cuando ya se hayan obtenido 3 imagenes de 20 especies
    if (species_count == 21):
        break
            
df_eol = pd.DataFrame(rows_list)
df_eol

,Specie,Url
0,Senna alata,https://content.eol.org/data/media/7e/e3/ec/54...
1,Senna alata,https://content.eol.org/data/media/7e/e3/ec/54...
2,Senna alata,https://content.eol.org/data/media/7e/e3/ec/54...
3,Momordica charantia,https://content.eol.org/data/media/00/22/ea/8....
4,Momordica charantia,https://content.eol.org/data/media/00/22/ea/8....
5,Momordica charantia,https://content.eol.org/data/media/00/22/ea/8....
6,Cocos nucifera,https://content.eol.org/data/media/80/01/5b/54...
7,Cocos nucifera,https://content.eol.org/data/media/80/01/5b/54...
8,Cocos nucifera,https://content.eol.org/data/media/80/01/5b/54...
9,Sphagneticola trilobata,https://content.eol.org/data/media/80/83/3c/54...


In [89]:
print('La cantidad de especies sin foto son: ',20-((df_eol.Specie.value_counts().sum())/3))

La cantidad de especies sin foto son:  3.0


In [90]:
L2_1 = L.sample(3, replace=True)
L2_1

,kingdom,phylum,order,family,genus,species,scientificName
0,Plantae,Tracheophyta,Aquifoliales,Aquifoliaceae,Ilex,Ilex lamprophylla,Ilex lamprophylla Standl.
223,Plantae,Tracheophyta,Gentianales,Gentianaceae,Voyria,Voyria alba,Voyria corymbosa subsp. alba (Standl.) Ruyters...
161,Plantae,Tracheophyta,Gentianales,Apocynaceae,Allamanda,Allamanda blanchetii,Allamanda blanchetii A.DC.


Checkeo de especies entre listas

In [92]:
spec1 = list(df_eol['Specie'].unique())
spec2 = list(L2_1.species.unique())

check =  any(item in spec2 for item in spec1)
 
if check is True:
    print("La lista si contiene algunas especies de la primera lista")   
else :
    print("Las listas no contiene especies iguales.")

Las listas no contiene especies iguales.


### Se realiza la busqueda de fotos en eol de las especies faltantes con este requerimiento

Se procede a guardar las imagenes en una **carpeta**

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
count = 0
for url, name in zip(L_20['identifier'], L_20['species']):
    n = os.path.exists("/content/drive/MyDrive/imagenes/"+name+'.jpg')
    if n:
        count +=1
        if (count < 3):
            urllib.request.urlretrieve(str(url), "/content/drive/MyDrive/imagenes/"+name+'_'+str(count)+".jpg")
    else:
        count = 0
        urllib.request.urlretrieve(str(url), "/content/drive/MyDrive/imagenes/"+name+".jpg")

file_list = glob.glob(os.path.join(os.getcwd(), "/content/drive/MyDrive/imagenes/", "*.jpg"))
print("La cantidad de imagenes descargadas son: ",len(file_list))

La cantidad de imagenes descargadas son:  34


## Prueba del identificador de especies de Plantnet

Para la evaluación se emplean las fotografías obtenidas en el proceso anterior.<br>

A continuación, una lista de las librerías utilizadas en esta seccion:

In [ ]:
#URL de descarga del webdriver: https://chromedriver.chromium.org/

#Se cargan las librerias respectivas
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

En las siguientes líneas de código se realizan una serie de procesos automatizados que corresponden con la técnica de webscrapping los cuales corresponden con: ejecución del webdriver en cuyo caso abre el GoogleChrome, posteriormente se busca el sitio web de plantnet que corresponde con la app de identificación y, en este sitio se utiliza el boton de subida de imagenes para su respectiva identificación, con esto se obtiene un resultado del cual únicamente se copia la información de la primera fila con el que se crea un dataframe que incluye como columnas el nombre del archivo evaluado, la nombre científico, la familia, el nombre común y el porcentaje de precisión que se obtuvo con la identificación que realiza la app.<br>

In [ ]:
#Se ejecuta el webdriver para chrome
driver = webdriver.Chrome(executable_path="/content/drive/MyDrive/webdriver/chromedriver.exe")
driver.implicitly_wait(0.5)
driver.maximize_window()

#Se especifica la pagina web
driver.get("https://identify.plantnet.org/")

#Identificar el elemento donde cargar la foto
s = driver.find_element_by_xpath("//input[@type='file']")

#Se crea un dataset con los datos obtenidos
fname = list()
nsc = list()
fam = list()
ncom = list()
prec = list()
pic = list()

for i in range(0, len(file_list)):
    s.send_keys(str(file_list[i]))
    fname.append(os.path.basename(file_list[i]))
    time.sleep(40)
    #Selecciona los primeros datos de la identificacion
    text_plant = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div/div/div/div[1]/div[1]/div')
    text_plant = text_plant.text
    text_plant = text_plant.split('\n')
    nsc.append(text_plant[0])
    fam.append(text_plant[1])
    if(len(text_plant) == 4):
        ncom.append(text_plant[2])
    else:
        ncom.append("ND")
    prec.append(text_plant[-1])
    pic.append(str(file_list[i]))

df = pd.DataFrame({'Foto': fname,'Nombre_cientifico': nsc, 'Familia': fam, 'Nombre_comun': ncom, 'Precision':prec})
df['Precision'] = [float(v.replace('%','').strip()) for v in df.Precision]

driver.quit()

A partir del procedimiento anterior se construye la siguiente tabla:<br>

In [ ]:
df

,Foto,Nombre_cientifico,Familia,Nombre_comun,Precision
0,a.jpg,Cattleya schilleriana Rchb.f.,Orchidaceae,ND,54.25
1,b.jpg,Opuntia monacantha (Willd.) Haw.,Cactaceae,Cochineal prickly-pear,53.78
2,c.jpg,Tecoma stans (L.) Juss. ex Kunth,Bignoniaceae,Trumpet-flower,7.64
3,d.jpg,Thuja occidentalis L.,Cupressaceae,Northern white-cedar,37.06
4,e.jpg,Coffea arabica L.,Rubiaceae,Arabian coffee,32.81
5,f.jpg,Tulbaghia violacea Harv.,Amaryllidaceae,ND,46.45
6,g.jpg,Lycopodium clavatum L.,Lycopodiaceae,Ground-pine,70.32
7,h.jpg,Fuchsia triphylla L.,Onagraceae,ND,22.45
8,i.jpg,Sarracenia purpurea L.,Sarraceniaceae,Common pitcherplant,21.46
9,j.jpg,Eryngium lemmonii J.M. Coult. & Rose,Apiaceae,Chiricahua Mountain eryngo,56.90


Por último, con el fin de evaluar en términos globales la precisión que se obtuvo con la puesta a prueba de esta app, se aplica una consulta a la columna 'Precision' respecto de sus estadísticas descriptivas.<br>

In [ ]:
df['Precision'].describe()

count    13.000000
mean     42.153077
std      23.897118
min       2.190000
25%      22.450000
50%      46.450000
75%      56.900000
max      79.100000
Name: Precision, dtype: float64

De estas estadísticas se puede observar que se evaluaron 60 fotografias de 20 especies diferentes en las cuales se obtuvo una precision promedio de _%, siendo _% la precision mínima  y _% la máxima, tal y como es es posible apreciar en la tabla de resultados.<br>

## Fuentes: 

- https://api.gbif.org/v1/occurrence/
- http://eol.org/api/
- https://identify.plantnet.org/
- https://selenium-python.readthedocs.io/